# Justification and Discussion of negative potential (Table 1 and Table 2)

## Get filename

In [10]:
annotations_fn = '../data/annotations.tsv'

## Load and process annotations

In [11]:
import pandas as pd
import numpy as np
import plotly.express as px

# get annotations
annotations = pd.read_csv(annotations_fn, sep='\t').dropna(how='all', axis='columns')  # drop empty colums
annotations = annotations.rename({'Annotators 1 and 2': 'Annotator'}, axis=1) # rename columns
annotations = annotations[annotations.Title.notna()].reset_index(drop=True)  # drop empty rows
annotations = annotations[annotations.Complete.notna()].reset_index(drop=True)  # drop not done rows

# get all annotations
papers = list(annotations.Title.unique())
annotations['Negative potential?'] = annotations['Negative potential?'].fillna(0).apply(int)
annotations['Justification score'] = annotations['Justification score'].apply(lambda i: max(int(i),1) - 1)  # we deleted one category

# get annotations that were doubly annotated (by a pair of annotators)
for title in annotations.Title.unique():
    n_raters = len(annotations[annotations.Title == title])
    annotations.loc[annotations.Title == title, '# Annotations'] = n_raters
pairs_annotations = annotations.copy()[annotations['# Annotations'] == 2.0].drop(['# Annotations'], axis=1).reset_index(drop=True)
pairs_papers = list(pairs_annotations.Title.unique())

## Analyze

In [12]:
from collections import Counter
for analysis in ['Justification score', 'Negative potential?']:
    paper_scores = [annotations[annotations.Title == paper][analysis].max() for paper in papers]
    category_percents = (sorted({score: count * 100. / len(papers) for score, count in Counter(paper_scores).items()}.items()))
    print(analysis+'\n', '\n'.join([f'Category {cat}: {percent}%' for cat, percent in category_percents]) + '\n')

Justification score
 Category 0: 68.0%
Category 1: 17.0%
Category 2: 11.0%
Category 3: 4.0%

Negative potential?
 Category 0: 98.0%
Category 1: 1.0%
Category 2: 1.0%



## IRR

In [18]:
# IRR: Fleiss' weighted kappa
print("FLEISS' WEIGHTED KAPPA")
from sklearn.metrics import cohen_kappa_score
for category in ['Justification score', 'Negative potential?']:
    a1, a2 = zip(*[list(pairs_annotations.loc[pairs_annotations.Title == paper, category]) for paper in pairs_papers])
    kappa = cohen_kappa_score(a1, a2, weights='linear')
    print(f"'{category}' Fleiss' weighted kappa: {kappa:.2}")

FLEISS' WEIGHTED KAPPA
'Justification score' Fleiss' weighted kappa: 0.6
'Negative potential?' Fleiss' weighted kappa: 0.79
